In [2]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import itertools
import numpy as np
import scipy.sparse as sp
import dgl.data
from dgl.nn import GraphConv,SAGEConv,MaxPooling
import dgl.function as fn
from sklearn.metrics import roc_auc_score
from dgl import DropEdge,AddSelfLoop,GCNNorm,LaplacianPE
from torch_geometric.nn import PairNorm
from load_graph import simple_dataloader, Load_graph
from dgl.data import CoraGraphDataset, CiteseerGraphDataset, PPIDataset

import warnings
warnings.filterwarnings("ignore")

print(f"torch.__version__: {torch.__version__}")
print(f"dgl.__version__: {dgl.__version__}")

torch.__version__: 1.11.0+cu113
dgl.__version__: 0.8.2


create model 

In [3]:
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, 'mean')
        self.conv2 = SAGEConv(h_feats, h_feats, 'mean')
    
    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

In [4]:
class GCNBlock(nn.Module):
    def __init__(self, in_feats, out_feats, activation=None, pair_norm=False, add_self_loops=False, drop_edge=0):
        super(GCNBlock, self).__init__()
        self.conv = SAGEConv(in_feats, out_feats,'mean')
        self.dropedge = DropEdge(p=drop_edge)
        self.self_loops = AddSelfLoop()
        self.norm = PairNorm()
        self.activation = activation
        self.pair_norm = pair_norm
        self.add_self_loops = add_self_loops
        self.drop_edge = drop_edge

    def forward(self, g, in_feat):
        if self.drop_edge:
            g = self.dropedge(g)
            self.add_self_loops = True
        if self.add_self_loops:
            g = self.self_loops(g)
        h = self.conv(g, in_feat)
        if self.pair_norm:
            h = self.norm(h)
        if self.activation:
            h = self.activation(h)
        return h

class GraphSAGE(nn.Module):
    def __init__(self, in_feats, n_hidden, out_feats, n_layers, activation, dropout=0., 
                 pair_norm=False, add_self_loops=False, drop_edge=0):
        super(GraphSAGE, self).__init__()
        if dropout:
            self.dropout = nn.Dropout(p=dropout)
        else:
            self.dropout = 0.
        self.layers = nn.ModuleList()
        self.layer1 = GCNBlock(in_feats, n_hidden, activation, pair_norm=pair_norm,add_self_loops=add_self_loops, drop_edge=drop_edge)

        self.layers = nn.ModuleList()
        for i in range(n_layers - 1):
            self.layers.append(GCNBlock(n_hidden, n_hidden, activation, pair_norm=pair_norm,add_self_loops=add_self_loops, drop_edge=drop_edge))

        self.layers.append(GCNBlock(n_hidden, out_feats, activation=None, pair_norm=pair_norm,add_self_loops=add_self_loops,drop_edge=drop_edge))

    def forward(self, g, features):
        h = self.layer1(g, features)
        for idx, layer in enumerate(self.layers):
            if idx > 0 and self.dropout:
                h = self.dropout(h)
            h = layer(g, h)
        if self.dropout:
            h = self.dropout(h)
        return h

In [5]:
class DotPredictor(nn.Module):
    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            # Compute a new edge feature named 'score' by a dot-product between the
            # source node feature 'h' and destination node feature 'h'.
            g.apply_edges(fn.u_dot_v('h', 'h', 'score'))
            # u_dot_v returns a 1-element vector for each edge so you need to squeeze it.
            return g.edata['score'][:, 0]

In [6]:
class MLPPredictor(nn.Module):
    def __init__(self, h_feats):
        super().__init__()
        self.W1 = nn.Linear(h_feats * 2, h_feats)
        self.W2 = nn.Linear(h_feats, 1)

    def apply_edges(self, edges):
        h = torch.cat([edges.src['h'], edges.dst['h']], 1)
        return {'score': self.W2(F.relu(self.W1(h))).squeeze(1)}

    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            g.apply_edges(self.apply_edges)
            return g.edata['score']

In [7]:
def compute_loss(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])])
    return F.binary_cross_entropy_with_logits(scores, labels)

def compute_auc(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score]).detach().numpy()
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).detach().numpy()
    return roc_auc_score(labels, scores)

training

In [15]:
dataset = Load_graph('cora', 'edge')
g = dataset[1]

u, v = g.edges()

eids = np.arange(g.number_of_edges())
eids = np.random.permutation(eids)
test_size = int(len(eids) * 0.1)
train_size = g.number_of_edges() - test_size
test_pos_u, test_pos_v = u[eids[:test_size]], v[eids[:test_size]]
train_pos_u, train_pos_v = u[eids[test_size:]], v[eids[test_size:]]

adj = sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy())))
adj_neg = 1 - adj.todense() - np.eye(g.number_of_nodes())
neg_u, neg_v = np.where(adj_neg != 0)

neg_eids = np.random.choice(len(neg_u), g.number_of_edges())
test_neg_u, test_neg_v = neg_u[neg_eids[:test_size]], neg_v[neg_eids[:test_size]]
train_neg_u, train_neg_v = neg_u[neg_eids[test_size:]], neg_v[neg_eids[test_size:]]
train_g = dgl.remove_edges(g, eids[:test_size])

train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=g.number_of_nodes())
train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=g.number_of_nodes())

test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=g.number_of_nodes())
test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=g.number_of_nodes())

In [22]:
model = GraphSAGE(train_g.ndata['feat'].shape[1],  n_hidden=512, out_feats=128, n_layers=3, activation=F.relu, dropout=0., 
                 pair_norm=False, add_self_loops=True, drop_edge=0.)

# pred = MLPPredictor(16)
pred = DotPredictor()

optimizer = torch.optim.Adam(itertools.chain(model.parameters(), pred.parameters()), lr=0.0002)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', factor=0.1, patience=5,threshold=0.0001, verbose=True, min_lr=1e-8)
all_logits = []
best_auc = 0.
for e in range(200):
    # forward
    h = model(train_g, train_g.ndata['feat'])
    train_pos_score = pred(train_pos_g, h)
    train_neg_score = pred(train_neg_g, h)
    loss = compute_loss(train_pos_score, train_neg_score)
    
    test_pos_score = pred(test_pos_g, h)
    test_neg_score = pred(test_neg_g, h)
    test_auc = compute_auc(test_pos_score, test_neg_score)
    
    if best_auc < test_auc:
        best_auc = test_auc
    
    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if e % 5 == 0:
        print('In epoch {}, loss: {:.3f}, test_auc: {:.3f}, best_auc: {:.3f}'.format(e, loss,  test_auc, best_auc))
        
#     if scheduler is not None:
#         scheduler.step(test_auc)

In epoch 0, loss: 111.905, test_auc: 0.511, best_auc: 0.511
In epoch 5, loss: 4.329, test_auc: 0.541, best_auc: 0.552
In epoch 10, loss: 1.246, test_auc: 0.546, best_auc: 0.552
In epoch 15, loss: 0.876, test_auc: 0.543, best_auc: 0.552
In epoch 20, loss: 0.765, test_auc: 0.545, best_auc: 0.552
In epoch 25, loss: 0.716, test_auc: 0.554, best_auc: 0.554
In epoch 30, loss: 0.698, test_auc: 0.565, best_auc: 0.565
In epoch 35, loss: 0.690, test_auc: 0.575, best_auc: 0.575
In epoch 40, loss: 0.686, test_auc: 0.583, best_auc: 0.583
In epoch 45, loss: 0.682, test_auc: 0.592, best_auc: 0.592
In epoch 50, loss: 0.680, test_auc: 0.600, best_auc: 0.600
In epoch 55, loss: 0.678, test_auc: 0.608, best_auc: 0.608
In epoch 60, loss: 0.676, test_auc: 0.616, best_auc: 0.616
In epoch 65, loss: 0.674, test_auc: 0.624, best_auc: 0.624
In epoch 70, loss: 0.672, test_auc: 0.631, best_auc: 0.631
In epoch 75, loss: 0.670, test_auc: 0.638, best_auc: 0.638
In epoch 80, loss: 0.667, test_auc: 0.646, best_auc: 0.6

ppi

In [10]:
dataset = PPIDataset(mode='train')
data = []
for i in range(len(dataset)):
    g = dataset[i]
    u, v = g.edges()
    eids = np.arange(g.number_of_edges())
    eids = np.random.permutation(eids)
    test_size = int(len(eids) * 0.1)
    train_size = g.number_of_edges() - test_size
    test_pos_u, test_pos_v = u[eids[:test_size]], v[eids[:test_size]]
    train_pos_u, train_pos_v = u[eids[test_size:]], v[eids[test_size:]]

    adj = sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy())))
    adj_neg = 1 - adj.todense() - np.eye(g.number_of_nodes())
    neg_u, neg_v = np.where(adj_neg != 0)

    neg_eids = np.random.choice(len(neg_u), g.number_of_edges())
    test_neg_u, test_neg_v = neg_u[neg_eids[:test_size]], neg_v[neg_eids[:test_size]]
    train_neg_u, train_neg_v = neg_u[neg_eids[test_size:]], neg_v[neg_eids[test_size:]]
    train_g = dgl.remove_edges(g, eids[:test_size])

    train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=g.number_of_nodes())
    train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=g.number_of_nodes())

    test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=g.number_of_nodes())
    test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=g.number_of_nodes())
    
    data.append([train_g, train_pos_g, train_neg_g, test_pos_g, test_neg_g])
    
loader = simple_dataloader(dataset=data)

In [24]:
data_dataset = PPIDataset(mode='train')
model = GraphSAGE(data_dataset[0].ndata['feat'].shape[1],  n_hidden=512, out_feats=128, n_layers=3, activation=F.relu, dropout=0., 
                 pair_norm=True, add_self_loops=True, drop_edge=0.)

# pred = MLPPredictor(16)
pred = DotPredictor()

optimizer = torch.optim.Adam(itertools.chain(model.parameters(), pred.parameters()), lr=0.001)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', factor=0.1, patience=5,threshold=0.0001, verbose=True, min_lr=1e-8)
all_logits = []
best_auc = 0.
for e in range(500):
    
    # forward
    losses=[]
    test_aucs=[]
    for train_g, train_pos_g, train_neg_g, test_pos_g, test_neg_g in loader:
        h = model(train_g, train_g.ndata['feat'])
        train_pos_score = pred(train_pos_g, h)
        train_neg_score = pred(train_neg_g, h)
        loss = compute_loss(train_pos_score, train_neg_score)
        losses.append(loss.item())

        test_pos_score = pred(test_pos_g, h)
        test_neg_score = pred(test_neg_g, h)
        test_auc = compute_auc(test_pos_score, test_neg_score)
        test_aucs.append(test_auc)
        
        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    test_auc = np.array(test_aucs).mean()
    loss = np.array(losses).mean()
    if best_auc < test_auc:
        best_auc = test_auc
    
    if e % 5 == 0:
        print('In epoch {}, loss: {:.3f}, test_auc: {:.3f}, best_auc: {:.3f}'.format(e, loss,  test_auc, best_auc))
        
#     if scheduler is not None:
#         scheduler.step(test_auc)

In epoch 0, loss: 0.674, test_auc: 0.645, best_auc: 0.645
In epoch 5, loss: 0.580, test_auc: 0.782, best_auc: 0.782
In epoch 10, loss: 0.558, test_auc: 0.806, best_auc: 0.806
In epoch 15, loss: 0.549, test_auc: 0.816, best_auc: 0.816
In epoch 20, loss: 0.544, test_auc: 0.822, best_auc: 0.822
In epoch 25, loss: 0.540, test_auc: 0.825, best_auc: 0.825
In epoch 30, loss: 0.539, test_auc: 0.827, best_auc: 0.827
In epoch 35, loss: 0.536, test_auc: 0.829, best_auc: 0.829
In epoch 40, loss: 0.535, test_auc: 0.830, best_auc: 0.830
In epoch 45, loss: 0.534, test_auc: 0.830, best_auc: 0.830
In epoch 50, loss: 0.533, test_auc: 0.831, best_auc: 0.831
In epoch 55, loss: 0.532, test_auc: 0.832, best_auc: 0.832
In epoch 60, loss: 0.531, test_auc: 0.833, best_auc: 0.833
In epoch 65, loss: 0.530, test_auc: 0.833, best_auc: 0.834
In epoch 70, loss: 0.531, test_auc: 0.832, best_auc: 0.834
In epoch 75, loss: 0.529, test_auc: 0.834, best_auc: 0.835
In epoch 80, loss: 0.528, test_auc: 0.835, best_auc: 0.835